## Парсинг данных

В качестве источника данных был выбран сайт auto.ru. Проведенный EDA тестовой выборки позволил сузить область поиска до автомобилей марки BMW. Кроме того, был обозначен круг искомых характеристик автомобилей. Сбор данных осуществлялся при помощи библиотеки requests. Сайт auto.ru формирует файл json по запросу пользователя. Работа с этим файлом велась при помощи библиотеки json.

In [3]:
import pandas as pd
import requests
import json
import numpy as np
from tqdm import tqdm

# Заголовки для запроса
headers = """Host: auto.ru
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0
Accept: */*
Accept-Language: ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3
Accept-Encoding: gzip, deflate, br
Referer: https://auto.ru/cars/bmw/all/?page=2
x-client-app-version: 202008.20.163652
x-page-request-id: 8c8bd543d7365432d1884e9d744fb341
x-client-date: 1598266403408
x-csrf-token: fb654da365f2d303219489b8d12e59ff0ea460e177a08bd3
x-requested-with: fetch
content-type: application/json
Origin: https://auto.ru
Content-Length: 78
Connection: keep-alive
Cookie: autoru_sid=a:g5f4369e32qpbrpn58uroi90qfru7trc.dc749a316e486c2c5f6738c2e9848555|1598253539292.604800.QbtGJ8en9-4l6O2E_5G7xA.NylRbaS3zRp634H09gtradAPyF4zFPrmzb4sq2FfvU0; autoruuid=g5f4369e32qpbrpn58uroi90qfru7trc.dc749a316e486c2c5f6738c2e9848555; X-Vertis-DC=myt; _ym_wasSynced=%7B%22time%22%3A1598253542576%2C%22params%22%3A%7B%22eu%22%3A0%7D%2C%22bkParams%22%3A%7B%7D%7D; gdpr=0; _ym_uid=1598253543561825513; _ym_d=1598266381; _ym_isad=2; _csrf_token=fb654da365f2d303219489b8d12e59ff0ea460e177a08bd3; suid=1836e4649b35b40e22e87197b4de3abc.149ec7567ff860e58429ecfb0c2e6049; from_lifetime=1598266381763; from=direct; yuidcs=1; yuidlt=1; yandexuid=130881971596628465; crookie=dsPRqHVK55jKRCz4kt4JrmrBoSRut6htZv53MgVz1SxI5fviYUa0Muy8xQRUIyyQnuWT1NatrSNPhQ4JH3OkiFkwLdM=; cmtchd=MTU5ODI1NTY1Njg4Nw==; cycada=V0yGd+bhYpjxQ/LSwk7N6iZvAMn4ifq0qKZiqZNOnE8=; _ym_visorc_22753222=b; _ym_visorc_148383=w; gids=; _ym_visorc_148422=w
DNT: 1""".strip().split("\n")

# Cоздадим словарь
dict_headers = {}
for header in headers:
    key, val = header.split(": ")
    dict_headers[key] = val


def take_data(mark, series, page, headers):
    """Функция принимает на вход марку автомобиля (mark), серию (series), номер страницы (page) 
       и заголовки (headers).Возвращает json c характеристиками автомобиля.
    """

    # Путь запроса
    url = 'https://auto.ru/-/ajax/desktop/listing/'

    # Параметры для запроса
    params = {
        'catalog_filter': [{"mark": mark, "model": series}],
        'section': "all",
        'category': "cars",
        'sort': "fresh_relevance_1-desc",
        'page': page
    }

    # Делаем post запрос на url
    response = requests.post(url, json=params, headers=dict_headers)
    return response.json()['offers']

Создадим пустой датафрейм для дальнейшего заполнения. Список колонок в основном взят из EDA тестовой выборки. 

In [9]:
result = pd.DataFrame(columns=["body_type", "brand", "color", "fuel_type",
                               "model_date_begin", "model_date_end", "name", "model_name",
                               "number_of_doors", "production_date", "vehicle_transmission",
                               "engine_displacement", "engine_power", "mileage", "complectation",
                               "gear_type", "steering_wheel", "not_damage", "owners",
                               "pts_origin", "custom_clear", "price"])

In [178]:
# Количество машин на одной странице сайта
cars_per_page = 37

# Словарь -  серия: количество авто на сайте
series_count = {"1ER": 781,
                "2ER": 230,
                "3ER": 4262,
                "4ER": 148,
                "5ER": 5265,
                "6ER": 415,
                "7ER": 1505,
                "8ER": 51,
                "2ACTIVETOURER": 12,
                "2GRANDTOURER": 17,
                "M1": 1,
                "M2": 7,
                "M3": 40,
                "M4": 32,
                "M5": 78,
                "M6": 26,
                "M8": 13,
                "I3": 55,
                "I8": 12,
                "X1": 1109,
                "X2": 82,
                "X3": 1767,
                "X3_M": 2,
                "X4": 394,
                "X4_M": 4,
                "X5": 3232,
                "X5_M": 122,
                "X6": 1490,
                "X6_M": 216,
                "X7": 296,
                "Z1": 1,
                "Z3": 8,
                "Z3M": 2,
                "Z4": 48
                }

Запустим сбор данных. Так как не для каждого авто имелись запрашиваемые данные, пришлось воспользоваться оператором try-catch. 

In [179]:
print("BMW auto info parsing start...")
for key, val in series_count.items():
    print("Series to parse:", key)
    for page in tqdm(range(1, val//cars_per_page+2)):
        data = take_data("BMW", key, page, dict_headers)
        for i in range(len(data)):
            to_append = []
            try:
                to_append.append(data[i]['vehicle_info']
                                 ['configuration']['human_name'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']['mark_info']['name'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]["color_hex"])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']
                                 ['tech_param']['engine_type'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']
                                 ['super_gen']['year_from'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']
                                 ['super_gen']['year_to'])
            except:
                to_append.append(2020)

            try:
                to_append.append(data[i]['vehicle_info']
                                 ['tech_param']['human_name'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']['model_info']['name'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']
                                 ['configuration']['doors_count'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['documents']['year'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']
                                 ['tech_param']['transmission'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']
                                 ['tech_param']['displacement'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']
                                 ['tech_param']['power'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['state']['mileage'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(str(data[i]['vehicle_info']['equipment']))
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']
                                 ['tech_param']['gear_type'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['vehicle_info']['steering_wheel'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['state']['state_not_beaten'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['documents']['owners_number'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['documents']['pts'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['documents']['custom_cleared'])
            except:
                to_append.append(np.nan)

            try:
                to_append.append(data[i]['price_info']['RUR'])
            except:
                to_append.append(np.nan)

            result.loc[len(result)] = to_append

  0%|                                                   | 0/41 [00:00<?, ?it/s]

BMW auto info parsing start...
Series to parse: 7ER


  0%|                                                    | 0/2 [00:00<?, ?it/s]

Series to parse: 8ER


  0%|                                                    | 0/1 [00:00<?, ?it/s]

...

  0%|                                                    | 0/1 [00:00<?, ?it/s]

Series to parse: Z3M


  0%|                                                    | 0/2 [00:00<?, ?it/s]

Series to parse: Z4


100%|████████████████████████████████████████████| 2/2 [00:07<00:00,  3.76s/it]


Удалим дубликаты записей перед сохранением данных, а также записи, в которых отсутствует цена (price)

In [203]:
result = result.drop_duplicates()
result.dropna(subset=['price'], inplace=True)

Сохраним полученные данные:

In [206]:
result.to_csv('raw_data.csv', index=False, header=True)

In [10]:
print(
    f'В результате работы программы удалось собрать данные о {result.shape[0]} автомобилях')

В результате работы программы удалось собрать данные о 18417 автомобилях
